# Use Naive Bayes to predict the News Category

In [2]:
import numpy as np
import random
import threading
import warnings
import os
import sys
import nltk
sys.path.append('../../common/')

from train_log import train_log

os.environ["CUDA_VISIBLE_DEVICES"]="1,2" 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.FATAL)

FILE_NAME = 'News_Category_Dataset_v2.json'

ckpt_dir = './ckpt_save/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
log_dir = './log/'
log = train_log(log_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

CONTINUE = 1
start_step = 150000

In [5]:
from NewsCategoryData import NewsCategory
from NewsCategoryData import LABEL_LIST
batch_size = 64
max_length = 100
data = NewsCategory(batch_size=batch_size,max_length=max_length)

max_recorder = data.max_recorder
word_dict = {}

for i in range(max_recorder):
    for word in data.data[i]:
        if word.lower() in word_dict:
            word_dict[word.lower()][data.label[i]] += 1
        else:
            word_dict[word.lower()] = [0 for i in range(len(LABEL_LIST))]
            word_dict[word.lower()][data.label[i]] += 1

58834
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
import csv

word_list = {}
word_matrix = []
index = 0
for word in word_dict:
    word_list[word] = index 
    word_matrix.append(word_dict[word])
    index += 1
    
word_matrix = np.asarray(word_matrix) 
word_matrix = np.divide(word_matrix,word_matrix.sum(axis=0))

fo = open("naive_bayes_word_matrix.csv", "w+")
writer = csv.writer(fo)

for key in word_list:
    writer.writerow([key]+list(word_matrix[word_list[key]]))
fo.close()     

In [7]:
word_matrix_2 = []
word_list_2 = {}
fo = open("naive_bayes_word_matrix.csv", "r+")
reader = csv.reader(fo)
index = 0
for row in reader:
    word_list_2[row[0]] = index
    index += 1
    m=[]
    for data in row[1:]:
        m.append(float(data))
    word_matrix_2.append(m)
word_matrix_2 = np.array(word_matrix_2)
print(len(word_list_2))
print(word_matrix_2.shape)

58834
(58834, 41)


In [7]:
import numpy as np

accuracy = 0
icount = 0
prediction = []
for title in data.data:
    y = np.array([1 for i in range(word_matrix.shape[1])], dtype=np.float32)
    x = [] 
    for word in title:
        x.append(word_matrix[word_list[word.lower()]])
    y = np.asarray(x).prod(axis=0)   
    #print("y=",y)
    prediction.append(y.argmax())
    #print(prediction,data.label[icount])
    if y.argmax() == data.label[icount]:
        accuracy += 1
    icount += 1
print("Accuracy rate = %.2f %%"%(accuracy/len(data.label)*100))        

Accuracy rate = 77.40 %
